In [1]:
from glob import glob
import pandas as pd
import numpy as np
from random import choice, sample
from tensorflow.keras.utils import plot_model

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract
from keras.models import Model
from keras.optimizers import Adam
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace 
import cv2
from collections import defaultdict


In [2]:
train_triplets = pd.read_csv('TSKinFace/train_triplets.csv')


Fathers_train = train_triplets.F.values     # No. of fathers
Mothers_train = train_triplets.M.values     # No. of mothers
Children_train = train_triplets.C.values    # No. of children

train_folders_path = "TSKinFace\\train-faces\\train-faces\\"
all_train_images = glob(train_folders_path + "*/MID*/*.jpg")

print("\nNumber of Training Images:", len(all_train_images))
print("Sample Path of each Training Image:", all_train_images[0])

ppl = [x.split("\\")[-3] + "/" + x.split("\\")[-2] for x in all_train_images]

ppl = (set(ppl))


triplet_family_comb = list(zip(train_triplets.F.values, train_triplets.M.values, train_triplets.C.values))
triplet_family_comb = [x for x in triplet_family_comb if x[0] in ppl and x[1] in ppl and x[2] in ppl]

Fathers_train = [x for x in Fathers_train if x in ppl]
Mothers_train = [x for x in Mothers_train if x in ppl]
Children_train = [x for x in Children_train if x in ppl]

print("\nNumber of Families:", len(triplet_family_comb))
training_labels = [1]*len(triplet_family_comb)

print("\nSample 1 of Family Tuple:", triplet_family_comb[0])
print("Sample 1 Label of Training Triplet Tuple:", training_labels[0])

map_train_individual_to_images = defaultdict(list)

for x in all_train_images:
    map_train_individual_to_images[x.split(
        "\\")[-3] + "/" + x.split("\\")[-2]].append(x)

print("\nNumber of people in Trianing Data:",
    len(map_train_individual_to_images.keys()))
print("Sample of mapping each person to their images:",
    list(map_train_individual_to_images.items())[0])




Number of Training Images: 15845
Sample Path of each Training Image: TSKinFace\train-faces\train-faces\F0001\MID1\P00001_face0.jpg

Number of Families: 1209

Sample 1 of Family Tuple: ('F0002/MID1', 'F0002/MID2', 'F0002/MID3')
Sample 1 Label of Training Triplet Tuple: 1

Number of people in Trianing Data: 3020
Sample of mapping each person to their images: ('F0001/MID1', ['TSKinFace\\train-faces\\train-faces\\F0001\\MID1\\P00001_face0.jpg', 'TSKinFace\\train-faces\\train-faces\\F0001\\MID1\\P00002_face0.jpg', 'TSKinFace\\train-faces\\train-faces\\F0001\\MID1\\P00003_face0.jpg', 'TSKinFace\\train-faces\\train-faces\\F0001\\MID1\\P00004_face1.jpg', 'TSKinFace\\train-faces\\train-faces\\F0001\\MID1\\P00007_face1.jpg', 'TSKinFace\\train-faces\\train-faces\\F0001\\MID1\\P00008_face4.jpg'])


In [3]:
val_triplets = pd.read_csv('TSKinFace/val_triples_competition_with_label.csv')


Fathers_val = val_triplets.F.values
Mothers_val = val_triplets.M.values
Children_val = val_triplets.C.values

val_folders_path = "TSKinFace\\val-faces\\val-faces\\"
all_val_images = glob(val_folders_path + "*/MID*/*.jpg")

print("\nNumber of validation Images:", len(all_val_images))
print("Sample Path of each validation Image:", all_val_images[0])

ppl = [x.split("\\")[-3] + "/" + x.split("\\")[-2] + "/" + x.split("\\")[-1] for x in all_val_images]

Fathers_val = [x for x in Fathers_val if x in ppl]
Mothers_val = [x for x in Mothers_val if x in ppl]
Children_val = [x for x in Children_val if x in ppl]

triplet_val = list(
    zip(val_triplets.F.values, val_triplets.M.values, val_triplets.C.values))
triplet_val = [x for x in triplet_val if x[0] in ppl and x[1] in ppl and x[2] in ppl]

print("\nNumber of validation Pairs:", len(triplet_val))

validation_labels = list(val_triplets.label.values)
validation_labels = [int(x) for x in validation_labels]

print("\nSample 1 of validation Triplet Tuple:", triplet_val[0])
print("Sample 1 Label of validation Triplet Tuple:", validation_labels[0])

print("\nSample 2 of validation Triplet Tuple:", triplet_val[1])
print("Sample 2 Label of validation Triplet Tuple:", validation_labels[1])

map_val_individual_to_images = defaultdict(list)

for x in all_val_images:
    map_val_individual_to_images[x.split("\\")[-3] + "\\" + x.split("\\")[-2]].append(x)

print("\nNumber of people in validation Data:", len(map_val_individual_to_images.keys()))
print("Sample of mapping each person to their images:", list(map_val_individual_to_images.items())[0])



Number of validation Images: 5045
Sample Path of each validation Image: TSKinFace\val-faces\val-faces\F0007\MID1\P00073_face2.jpg

Number of validation Pairs: 3568

Sample 1 of validation Triplet Tuple: ('F0290/MID5/P03086_face7.jpg', 'F0290/MID1/P03087_face0.jpg', 'F0652/MID1/P03819_face3.jpg')
Sample 1 Label of validation Triplet Tuple: 0

Sample 2 of validation Triplet Tuple: ('F0879/MID2/P09279_face0.jpg', 'F0879/MID1/P09279_face3.jpg', 'F0879/MID5/P09276_face4.jpg')
Sample 2 Label of validation Triplet Tuple: 1

Number of people in validation Data: 966
Sample of mapping each person to their images: ('F0007\\MID1', ['TSKinFace\\val-faces\\val-faces\\F0007\\MID1\\P00073_face2.jpg', 'TSKinFace\\val-faces\\val-faces\\F0007\\MID1\\P00074_face3.jpg', 'TSKinFace\\val-faces\\val-faces\\F0007\\MID1\\P00075_face0.jpg', 'TSKinFace\\val-faces\\val-faces\\F0007\\MID1\\P00076_face5.jpg', 'TSKinFace\\val-faces\\val-faces\\F0007\\MID1\\P00077_face12.jpg', 'TSKinFace\\val-faces\\val-faces\\F0007\

In [4]:
def read_img(path):
    # print('\n',8)
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_LINEAR)

    # print('\n',9)
    img = np.array(img, dtype=object).astype(np.float64)
    # print('\n',10)
    return preprocess_input(img, version=2)


def gen(list_tuples, person_to_images_map, F, M, C, batch_size=16):
    # print('\n',1)
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)  # postive samples
        while len(batch_tuples) < batch_size:
            p1 = choice(F)
            p2 = choice(M)
            p3 = choice(C)

            if (p1, p2, p3) not in list_tuples and p1 != p2  and p2 != p3 and p1 != p3:
                batch_tuples.append((p1, p2, p3))
                labels.append(0)  # negative samples
        # print('\n', batch_tuples)
        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = [read_img(x) for x in X1]
        # shapeX1 = [x.shape for x in X1]
        # print(shapeX1, len(shapeX1))
        X1 = np.array(X1)
        # print('\n',2)

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = [read_img(x) for x in X2]
        # shapeX2 = [x.shape for x in X2]
        # print(shapeX2, len(shapeX2))
        X2 = np.array(X2)
        # print('\n',3)

        X3 = [choice(person_to_images_map[x[2]]) for x in batch_tuples]
        X3 = [read_img(x) for x in X3]
        # shapeX3 = [x.shape for x in X3]
        # print(shapeX3, len(shapeX3))
        X3 = np.array(X3)
        # print('\n',4)
        labels = np.asarray(labels).astype('float32').reshape((-1, 1))
        # print('\n',5)
        yield [X1, X2, X3], labels


def gen_val(triplet_val, validation_labels, batch_size=16):
    while True:
        val = list(zip(triplet_val, validation_labels))
        val0 = [x for x in val if x[1] == 0]
        val1 = [x for x in val if x[1] == 1]
        batch_tuples0 = sample(val0, batch_size//2)
        batch_tuples1 = sample(val1, batch_size//2)
        batch_tuples = batch_tuples0 + batch_tuples1
        # print(batch_tuples)
        X1 = [read_img(val_folders_path + x[0][0]) for x in batch_tuples]
        X1 = np.array(X1)
        X2 = [read_img(val_folders_path + x[0][1]) for x in batch_tuples]
        X2 = np.array(X2)
        X3 = [read_img(val_folders_path + x[0][2]) for x in batch_tuples]
        X3 = np.array(X3)
        labels = [0]*(batch_size//2) + [1]*(batch_size//2)
        labels = np.asarray(labels).astype('float32').reshape((-1, 1))
        # print(labels)
        yield [X1, X2, X3], labels


In [5]:
# Model

def baseline_model():
    # print('\n',7)
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))
    input_3 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for x in base_model.layers[:-3]:
        x.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)
    x3 = base_model(input_3)

    x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
    x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])
    x3 = Concatenate(axis=-1)([GlobalMaxPool2D()(x3), GlobalAvgPool2D()(x3)])

    x4 = Subtract()([x1, x3])
    x4 = Multiply()([x4, x4])

    x5 = Subtract()([x2, x3])
    x5 = Multiply()([x5, x5])

    x13 = Multiply()([x1, x3])
    x13 = Concatenate(axis=-1)([x13, x4])

    x23 = Multiply()([x2, x3])
    x23 = Concatenate(axis=-1)([x23, x5])

    x = Concatenate(axis=-1)([x13, x23])

    x = Dense(100, activation="relu")(x)
    x = Dropout(0.01)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model([input_1, input_2, input_3], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.0001))

    model.summary()

    plot_model(model, to_file='model_plot.png',
               show_shapes=True, show_layer_names=True)
    return model


In [6]:
file_path = "vgg_face.h5"

checkpoint = ModelCheckpoint(
    file_path, monitor='acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(
    monitor="val_acc", mode="max", factor=0.1, patience=3, verbose=1)

callbacks_list = [checkpoint, reduce_on_plateau]

model = baseline_model()
# model.load_weights(file_path)
model.fit(gen(triplet_family_comb, map_train_individual_to_images, Fathers_train, Mothers_train, Children_train, batch_size=16),
          validation_data=gen_val(triplet_val, validation_labels, batch_size=16), epochs=30, verbose=1,
          callbacks=callbacks_list, steps_per_epoch=50, validation_steps=10)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                            